<a href="https://colab.research.google.com/github/alabamahanna123/Arthur/blob/main/CompareLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")
billsum_df = pd.DataFrame(billsum).sample(10, random_state=278487)


In [ ]:
from langchain.llms import OpenAI, Cohere
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

gpt3 = OpenAI(temperature=0.0, max_tokens=100)
command = Cohere(temperature=0.0, max_tokens=100)

prompt_template = PromptTemplate(
	input_variables=["text"],
	template="""
	You are an expert summarizer of text. A good summary
	captures the most important information in the text and doesnt focus too much on small details.
	Text: {text}
	Summary:
	"""
)

gpt3_chain = LLMChain(llm=gpt3, prompt=prompt_template)
command_chain = LLMChain(llm=command, prompt=prompt_template)

# generate summaries with truncated text
gpt3_summaries = [gpt3_chain.run(bill[:3000]) for bill in billsum_df.text]
command_summaries = [command_chain.run(bill[:3000]) for bill in billsum_df.text]

In [ ]:
from arthur_bench.run.testsuite import TestSuite
my_suite = TestSuite(
	"congressional_bills",
	"summary_quality",
	input_text_list=list(billsum_df.text),
	reference_output_list=list(billsum_df.summary)

In [ ]:
my_suite.run("gpt3_summaries", candidate_output_list=gpt3_summaries)
my_suite.run("command_summaries", candidate_output_list=command_summaries)